# Red neuronal Fully-Connected para Clasificación binaria



##### 1. Importaciones

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import logging


##### 2. Ajuste detalles mensajes de errores

In [2]:
logging.getLogger("tensorflow").setLevel(logging.ERROR)

##### 3. Importación DB (IMDB)

In [3]:
from tensorflow.keras.datasets import imdb


##### 4. Carga de datos

In [4]:
# se divide los datos para entrenar modelo y para testear rendimiento del modelo entrenado
(train_data, train_labels),(test_data, test_labels)=imdb.load_data(num_words=10000)


##### 5. Función para contar frecuencia palabras

In [9]:
# Devuelve la frecuencia
def to_vector(review):
    x=np.zeros(10000)
    for n in review:
        x[n]+=1.
        return x

##### 6. Conversión de datos

In [10]:
# Conversión de datos para entrenamiento modelo.
X_train = np.zeros((25000, 10000))
for i, x in enumerate(train_data):
    X_train[i]=to_vector(x)


In [12]:
# Conversión de datos para evalución modelo entrenamiento.
X_test =np.zeros((25000, 10000))
for i, x in enumerate(test_data):
    X_test[i]= to_vector(x)
    

##### 7. División conjunto de entrenamiento

In [ ]:
#División COnjunto de entrea en un 70% y un 30%  ------Finalizar
rand_idx 0 np.random.permutation(25000)